# Consolidation with DQN

In [1]:
"""
Gosssip-based virual machine consolidation in a cloud environment.
"""

'\nGosssip-based virual machine consolidation in a cloud environment.\n'

#### Load depenedancies

In [2]:
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np
import random
import math
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os

Using TensorFlow backend.


# Define the environment

In [3]:
class ConsolidationEnv(gym.Env):
    """
    Description:
        A datacenter has many virtual machines(VM). VMs need to get consolidated into some of the physical machines so some otheer physical machines can be turned off.
    Source:
        This environment corresponds to gosssip-based VM consolidation
    Observation: 
        Type: Box(2)
        Num	Observation                 Min         Max
        0	CPU Utilization            0.0            100.0
        1	Memory Utilization         0.0            100.0
        
    Actions:
        Type: Discrete(2)
        Num	Action
        0	Low 
        1	Medium
        2	High
        3	xHigh
        4	2xHigh
        5	3xHigh
        6	4xhigh
        7	5xHigh
        8	Overload
        
        
    Rewards-out:
        In sender mode, rewards are given to a PM to move out its VMs so that it can switch off
        Level	Reward
        0	1000
        1	900
        2	800
        3	700
        4	600
        5	500
        6	400
        7	300
        8	200
        
    Rewards-in:
        In recipient mode, rewards are given to avoid SLA violation. It occurs when a PM moves to an overload state
        Level	Reward
        0	100
        1	100
        2	100
        3	100
        4	100
        5	100
        6	100
        7	100
        8	-2000
        
    Starting State:
        All observations are assigned a uniform random value between 0,100
    Episode Termination:
        Pole Angle is more than ±12°
        Cart Position is more than ±2.4 (center of the cart reaches the edge of the display)
        Episode length is greater than 200
        Solved Requirements
        Considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials.
    """
    
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5 # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = 'euler'

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.cpu_min_utilization = 0.0
        self.cpu_max_utilization = 100.0
        self.memory_min_utilization = 0.0
        self.memory_max_utilization = 100.0

        # Angle limit set to 2 * theta_threshold_radians so failing observation is still within bounds
        low = np.array([
            self.cpu_min_utilization,
            self.memory_min_utilization])
        
        high = np.array([
            self.cpu_max_utilization,
            self.memory_max_utilization])

        self.action_space = spaces.Discrete(9) # actions
        self.observation_space = spaces.Box(low, high, dtype=np.float32) # states
        
        self.reward_out = {
            0 : 1000,
            1 : 900,
            2 : 800,
            3 : 700,
            4 : 600,
            5 : 500,
            6 : 400,
            7 : 300,
            8 : 200
        }
        
        self.reward_in = {
            0 : 100,
            1 : 100,
            2 : 100,
            3 : 100,
            4 : 100,
            5 : 100,
            6 : 100,
            7 : 100,
            8 : -2000
        }
        print("Reward {}".format(self.reward_out))

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        x, x_dot, theta, theta_dot = state
        force = self.force_mag if action==1 else -self.force_mag
        costheta = math.cos(theta)
        sintheta = math.sin(theta)
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta* temp) / (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc  = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        if self.kinematics_integrator == 'euler':
            x  = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else: # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x  = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot
        self.state = (x,x_dot,theta,theta_dot)
        done =  x < -self.x_threshold \
                or x > self.x_threshold \
                or theta < -self.theta_threshold_radians \
                or theta > self.theta_threshold_radians
        done = bool(done)

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn("You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.")
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=0.0, high=100.0, size=(2,))
        self.steps_beyond_done = None
        return np.array(self.state)


In [4]:
env = ConsolidationEnv() # initialise environment

Reward {0: 1000, 1: 900, 2: 800, 3: 700, 4: 600, 5: 500, 6: 400, 7: 300, 8: 200}


In [5]:
state_size = env.observation_space.shape[0]
state_size

2

In [6]:
action_size = env.action_space.n
action_size

9

In [7]:
batch_size = 32

In [8]:
n_episodes = 1000 # n games we want agen to play

In [9]:
output_dir = 'model_outpu/cartpole/'

In [10]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [11]:
class DQNAgentSender:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue for sender mode memory; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later
        
    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [12]:
class DQNAgentRecipient:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue for sender mode memory; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later
        
    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [13]:
sender_agent = DQNAgentSender(state_size, action_size) # initialise sender agent
recipient_agent = DQNAgentRecipient(state_size, action_size) # initialise recipient agent

In [14]:
done = False
for e in range(n_episodes): # iterate over new episodes of the game
    print("episode: {}/{}".format(e,n_episodes))
    
    state = env.reset() # reset state at start of each new episode of the game    
    state = np.reshape(state, [1, state_size])
    print("State[CPU, Memory] => {}".format(state))
    
    sender_action = sender_agent.act(state) # action is to select to migrate out some of the VMs from 0 to 8
    sender_reward = env.reward_out[sender_action]
    print("Sender Action:{}, Sender Reward:{}".format(sender_action,sender_reward))
    
    recipient_action = recipient_agent.act(state) # action is to select to migrate out some of the VMs from 0 to 8
    recipient_reward = env.reward_in[recipient_action]
    print("Recipient Action:{}, Recipient Reward:{}".format(recipient_action,recipient_reward))
    
#     next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
#     reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
#     next_state = np.reshape(next_state, [1, state_size])
#     agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
#     state = next_state # set "current state" for upcoming iteration to the current next state 
    
#     for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
#         action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
#         next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
#         reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
#         next_state = np.reshape(next_state, [1, state_size])
#         agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
#         state = next_state # set "current state" for upcoming iteration to the current next state        
#         if done: # episode ends if agent drops pole or we reach timestep 5000
#             print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
#                   .format(e, n_episodes, time, agent.epsilon))
#             break # exit loop
#     if len(agent.memory) > batch_size:
#         agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        sender_agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")
        recipient_agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")


episode: 0/1000
State[CPU, Memory] => [[49.30105211 71.6277584 ]]
Action:7, Sender Reward:300
Action:2, Recipient Reward:100
episode: 1/1000
State[CPU, Memory] => [[75.01307235 71.98909135]]
Action:4, Sender Reward:600
Action:4, Recipient Reward:100
episode: 2/1000
State[CPU, Memory] => [[72.00684591 89.1524487 ]]
Action:7, Sender Reward:300
Action:8, Recipient Reward:-2000
episode: 3/1000
State[CPU, Memory] => [[29.78692484 11.28682124]]
Action:3, Sender Reward:700
Action:6, Recipient Reward:100
episode: 4/1000
State[CPU, Memory] => [[19.32111101 64.79040249]]
Action:2, Sender Reward:800
Action:6, Recipient Reward:100
episode: 5/1000
State[CPU, Memory] => [[76.83592833 62.76008497]]
Action:2, Sender Reward:800
Action:7, Recipient Reward:100
episode: 6/1000
State[CPU, Memory] => [[69.55319736 85.94556595]]
Action:4, Sender Reward:600
Action:0, Recipient Reward:100
episode: 7/1000
State[CPU, Memory] => [[88.49322621 71.89693529]]
Action:0, Sender Reward:1000
Action:0, Recipient Reward:1

episode: 151/1000
State[CPU, Memory] => [[60.70217165 61.30186127]]
Action:6, Sender Reward:400
Action:4, Recipient Reward:100
episode: 152/1000
State[CPU, Memory] => [[46.87183319 74.13715868]]
Action:8, Sender Reward:200
Action:2, Recipient Reward:100
episode: 153/1000
State[CPU, Memory] => [[71.5157193   1.63582347]]
Action:6, Sender Reward:400
Action:7, Recipient Reward:100
episode: 154/1000
State[CPU, Memory] => [[94.43991253 38.96902857]]
Action:1, Sender Reward:900
Action:4, Recipient Reward:100
episode: 155/1000
State[CPU, Memory] => [[66.78467039 76.83311377]]
Action:5, Sender Reward:500
Action:6, Recipient Reward:100
episode: 156/1000
State[CPU, Memory] => [[ 6.85862236 94.38085443]]
Action:0, Sender Reward:1000
Action:6, Recipient Reward:100
episode: 157/1000
State[CPU, Memory] => [[39.64330641  5.78056308]]
Action:2, Sender Reward:800
Action:5, Recipient Reward:100
episode: 158/1000
State[CPU, Memory] => [[57.08449616 75.45685375]]
Action:8, Sender Reward:200
Action:2, Reci

Action:6, Sender Reward:400
Action:7, Recipient Reward:100
episode: 352/1000
State[CPU, Memory] => [[68.4312338  73.70912324]]
Action:1, Sender Reward:900
Action:4, Recipient Reward:100
episode: 353/1000
State[CPU, Memory] => [[91.05761129 16.34407276]]
Action:8, Sender Reward:200
Action:2, Recipient Reward:100
episode: 354/1000
State[CPU, Memory] => [[53.87078243 12.96883626]]
Action:5, Sender Reward:500
Action:6, Recipient Reward:100
episode: 355/1000
State[CPU, Memory] => [[51.65522552 49.70796191]]
Action:0, Sender Reward:1000
Action:1, Recipient Reward:100
episode: 356/1000
State[CPU, Memory] => [[95.85070786 56.81982735]]
Action:3, Sender Reward:700
Action:4, Recipient Reward:100
episode: 357/1000
State[CPU, Memory] => [[43.81874743 35.55053353]]
Action:2, Sender Reward:800
Action:8, Recipient Reward:-2000
episode: 358/1000
State[CPU, Memory] => [[61.3559479  47.81052932]]
Action:7, Sender Reward:300
Action:2, Recipient Reward:100
episode: 359/1000
State[CPU, Memory] => [[84.6793

episode: 601/1000
State[CPU, Memory] => [[70.69180944 13.13154529]]
Action:1, Sender Reward:900
Action:7, Recipient Reward:100
episode: 602/1000
State[CPU, Memory] => [[65.08312923 22.26963479]]
Action:5, Sender Reward:500
Action:8, Recipient Reward:-2000
episode: 603/1000
State[CPU, Memory] => [[62.66814341 54.82545868]]
Action:4, Sender Reward:600
Action:1, Recipient Reward:100
episode: 604/1000
State[CPU, Memory] => [[12.93685599 31.34710154]]
Action:5, Sender Reward:500
Action:2, Recipient Reward:100
episode: 605/1000
State[CPU, Memory] => [[62.88546073 86.47051312]]
Action:1, Sender Reward:900
Action:6, Recipient Reward:100
episode: 606/1000
State[CPU, Memory] => [[80.40186459 39.03216423]]
Action:5, Sender Reward:500
Action:2, Recipient Reward:100
episode: 607/1000
State[CPU, Memory] => [[88.4998392 23.6234352]]
Action:5, Sender Reward:500
Action:7, Recipient Reward:100
episode: 608/1000
State[CPU, Memory] => [[52.4626569  61.26141738]]
Action:3, Sender Reward:700
Action:7, Recip

episode: 801/1000
State[CPU, Memory] => [[98.50482408  1.17488943]]
Action:4, Sender Reward:600
Action:2, Recipient Reward:100
episode: 802/1000
State[CPU, Memory] => [[86.25157108 66.29281568]]
Action:7, Sender Reward:300
Action:2, Recipient Reward:100
episode: 803/1000
State[CPU, Memory] => [[46.19797172 96.88067296]]
Action:6, Sender Reward:400
Action:7, Recipient Reward:100
episode: 804/1000
State[CPU, Memory] => [[78.45014248  1.27154584]]
Action:8, Sender Reward:200
Action:3, Recipient Reward:100
episode: 805/1000
State[CPU, Memory] => [[63.61972902 56.48703928]]
Action:5, Sender Reward:500
Action:4, Recipient Reward:100
episode: 806/1000
State[CPU, Memory] => [[48.82959964 75.11124371]]
Action:0, Sender Reward:1000
Action:2, Recipient Reward:100
episode: 807/1000
State[CPU, Memory] => [[50.53973115 18.0161536 ]]
Action:0, Sender Reward:1000
Action:8, Recipient Reward:-2000
episode: 808/1000
State[CPU, Memory] => [[20.73497396 87.25059774]]
Action:3, Sender Reward:700
Action:4, R